In [7]:
import pandas as pd
import numpy as np
import json
import requests
import openpyxl

# SLEEP QUALITY AND DURATION DATA

In [8]:
# Importing the two datasets
country_abbrevs = pd.read_csv('data/country_abbrev_list.csv')

re = requests.get('https://sandman.sleepcycle.com/data/per-country-stats')
country_sleep_data = pd.DataFrame(re.json())

In [9]:
country_sleep_data

,country,avg_sleep_quality,avg_duration,avg_snore_duration,avg_bedtime,avg_wakeup,group_size
0,ca,0.7706280786149319,27203.393388256663,2658.828667280861,-0.010468966378401922,0.30500672811254403,28646
1,co,0.7297271999818904,25615.27579824156,1437.7857592261742,-0.03228318759677625,0.2644591619912697,2161
2,in,0.7060974741852611,25093.229276895945,2137.2825717013548,0.004082686733507055,0.2950252965085311,1134
3,il,0.7406121077804889,26042.273445212242,2051.836718594944,0.017049587070040857,0.31956485420204445,2026
4,se,0.786575637584524,27685.027529392293,3116.2711398811175,-0.007667250763352499,0.3134636193388775,24156
...,...,...,...,...,...,...,...
57,cy,0.7502196809844399,27078.2886002886,2816.03259211972,0.0043422235917116225,0.31831668729306556,693
58,jp,0.6784701596828577,24236.974481437002,3323.9126377116636,0.00410389190726472,0.2842500281637383,100711
59,gb,0.7840466475722139,27963.86005062065,2777.4097512704925,-0.00660779543906787,0.31820536414364314,65981
60,pe,0.7117202666169559,25158.92667509482,1714.1829438347725,-0.010349246817979626,0.2795599722538905,791


In [10]:
# Merging the two datasets
country_sleep_data['country'] = country_sleep_data['country'].str.upper()
sleep_by_country = pd.merge(country_sleep_data, country_abbrevs, left_on = 'country', right_on = 'Code').drop(['country', 'Code'], axis=1).set_index('Name')

In [11]:
# Converting all the strings to floats and converting sleep values to hours or minutes (from seconds)
sleep_by_country = sleep_by_country.astype(float)
sleep_by_country['avg_duration'] = round(sleep_by_country['avg_duration'] / 3600, 2)
sleep_by_country['avg_snore_duration'] = round(sleep_by_country['avg_snore_duration'] / 60, 2)

In [12]:
# Renaming columns appropriately and dropping unnecessary columns
sleep_by_country = sleep_by_country.rename(columns = {'avg_sleep_quality': 'Average Sleep Quality (0-1)', 'avg_duration': 'Average Sleep Duration (Hours)', 'avg_snore_duration': 'Average Snore Duration (Minutes)'})
sleep_by_country = sleep_by_country.drop(['avg_bedtime', 'avg_wakeup', 'group_size'], axis=1)

In [13]:
sleep_by_country

,Average Sleep Quality (0-1),Average Sleep Duration (Hours),Average Snore Duration (Minutes)
Name,,,
Canada,0.770628,7.56,44.31
Colombia,0.729727,7.12,23.96
India,0.706097,6.97,35.62
Israel,0.740612,7.23,34.20
Sweden,0.786576,7.69,51.94
...,...,...,...
Cyprus,0.750220,7.52,46.93
Japan,0.678470,6.73,55.40
United Kingdom,0.784047,7.77,46.29


# SOCIOECONOMIC DATA

In [16]:
# Importing and cleaning the dataset (removing some columns, making sure to only keep the most recent year for each country)
socioeconomic_data = pd.read_csv('data/raw_socioeconomic_data.csv', encoding = 'latin1')
socioeconomic_data = socioeconomic_data.drop(['unid', 'wbid', 'popshare', 'yrseduc'], axis=1)
socioeconomic_data = socioeconomic_data[socioeconomic_data['year'] == 2010]
socioeconomic_data = socioeconomic_data.set_index('country')

In [17]:
socioeconomic_data

,year,SES,gdppc
country,,,
Afghanistan,2010,5.676400,1662.8035
Angola,2010,21.247763,6492.1768
Albania,2010,74.860367,9927.1758
United Arab Emirates,2010,89.092285,57406.7380
Argentina,2010,79.750809,18794.2700
...,...,...,...
Vietnam,2010,38.222027,4486.2612
Yemen,2010,19.492294,4481.5547
South Africa,2010,71.227959,12028.9340


# WHO ENVIRONMENTAL DATA

In [18]:
# Importing the dataset
air_df = pd.read_csv('data/who_2022_air_quality.csv', encoding='latin1')

In [20]:
# Cleaning the dataset by getting only most recent year for each country and dropping unnecessary columns
air_df = air_df.loc[air_df.groupby('WHO Country Name')['Measurement Year'].idxmax()].set_index('WHO Country Name')
air_df = air_df.drop(['WHO Region', 'ISO3', 'Version of the database', 'Number and type of monitoring stations', 'Reference', 'Status', 'Unnamed: 15', 'Unnamed: 16'], axis=1)

In [21]:
air_df

,City or Locality,Measurement Year,PM2.5 (?g/m3),PM10 (?g/m3),NO2 (?g/m3),PM25 temporal coverage (%),PM10 temporal coverage (%),NO2 temporal coverage (%)
WHO Country Name,,,,,,,,
Afghanistan,Kabul,2019,119.77,NaN,NaN,18.0,NaN,NaN
Albania,Vlore,2019,10.32,NaN,NaN,NaN,NaN,NaN
Algeria,Algiers,2019,21.53,NaN,NaN,44.0,NaN,NaN
Andorra,Escaldes-Engordany,2019,NaN,24.58,31.01,NaN,NaN,92.180365
Argentina,Buenos Aires,2019,NaN,25.50,18.25,NaN,97.2,87.500000
...,...,...,...,...,...,...,...,...
United States of America,Albuquerque (Nm),2019,NaN,18.30,17.70,NaN,NaN,NaN
Uruguay,Montevideo,2018,12.00,19.00,NaN,NaN,NaN,NaN
Uzbekistan,Tashkent,2019,41.96,NaN,NaN,81.0,NaN,NaN


# Merging our 3 Datasets

In [32]:
merged_df = pd.merge(pd.merge(sleep_by_country, socioeconomic_data, left_index=True, right_index=True), air_df, left_index=True, right_index=True)

In [33]:
merged_df

,Average Sleep Quality (0-1),Average Sleep Duration (Hours),Average Snore Duration (Minutes),year,SES,gdppc,City or Locality,Measurement Year,PM2.5 (?g/m3),PM10 (?g/m3),NO2 (?g/m3),PM25 temporal coverage (%),PM10 temporal coverage (%),NO2 temporal coverage (%)
Canada,0.770628,7.56,44.31,2010,93.772118,40699.3550,Aamjiwnaang,2019,7.00,NaN,11.00,100.000000,NaN,100.000000
Colombia,0.729727,7.12,23.96,2010,59.636009,10900.5040,Barbosa,2019,10.49,NaN,NaN,17.000000,NaN,NaN
India,0.706097,6.97,35.62,2010,22.542639,4404.5376,Adilabad,2019,45.00,74.00,25.00,86.538461,100.000000,100.000000
Israel,0.740612,7.23,34.20,2010,86.517677,29599.7930,Afula,2015,NaN,63.50,NaN,NaN,94.400000,NaN
Sweden,0.786576,7.69,51.94,2010,91.959633,42942.5630,Älgarås,2019,NaN,NaN,1.20,NaN,NaN,93.401826
Spain,0.746307,7.46,43.95,2010,85.359718,32506.4340,A Coruna,2019,13.60,26.91,18.34,NaN,NaN,97.785388
Denmark,0.780589,7.65,51.91,2010,88.418976,43757.7380,Aalborg,2019,9.56,NaN,10.23,NaN,NaN,94.178082
Greece,0.729426,7.57,50.57,2010,85.653824,28726.0800,Ano Liosia,2019,NaN,NaN,21.07,NaN,NaN,85.696347
Slovakia,0.766138,7.50,47.71,2010,86.117645,25159.0780,Banska Bystrica,2019,13.92,21.06,19.09,NaN,NaN,95.719178
Estonia,0.787921,7.67,41.12,2010,87.317307,22740.9650,Kohtla-Jarve,2019,4.01,11.02,4.71,99.121005,99.121005,99.121005
